In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

block_size = 8 #
batch_size = 4 #
max_iterations = 10000 #
learning_rate = 3e-3 #
evaluate_iterations = 250 #

tensor([[ 0.3890,  0.1329, -0.0497, -0.0270,  0.1988]],
       grad_fn=<AddmmBackward0>)


In [ ]:
with open("wizard_of_oz.txt", 'r', encoding='utf-8') as f:
    text = f.read()
chars =  sorted(set(text))

vocabulary_size = len(chars)

print(chars)

In [ ]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

In [ ]:
# split the data into train and validation sets
n=int(0.8*len(data)) #0.8
train_data = data[:n]
val_data = data[n:]

# function to generate a batch of data
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    x,y = x.to(device), y.to(device)
    return x, y

x,y = get_batch('train')
print('inputs:')
print(x.shape)
print(x)
print('targets:')
print(y.shape)
print(y)

'''
https://www.youtube.com/watch?v=BTVUPhpXUd4&list=PLOZ8y37SEZTbf_57K1I4g7Jjrr6raTxbk&index=5
https://www.youtube.com/watch?v=Ip6NkFQ5DKw
https://www.youtube.com/watch?v=2FYzX2eW_aw

'''